# Amazon Deforestation Forecasting Using AI/ML and IoT Data
## AAI-530 Team 8 Final Project

Team 8:  Tyler Foreman

University of San Diego, Applied Artificial Intelligence

Date:  Feburary 11, 2023

GitHub Repository: https://github.com/t4ai/amazon-deforestation-forecasting